## Decision Tree & Random Forest Models

Rather than handling the data as a series of time series, in these models I create basic features from the time series data for baseline performance.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import duckdb
import re, gc

### Functions

In [2]:
# Create the rowwise sum of n days of observations for the specified range of columns; this range includes BOTH the start and end columns.
def n_day_sum(df, n=0, start_col=0, end_col=0, name_prefix=''):
    try:
        if (end_col - (start_col-1))%n != 0:
            raise ValueError('Column range must be evenly divisible by n.')
    except ValueError as e:
        print(f'{e}; DataFrame Unchanged; invalid column range. Start & end columns are included in range.')
        return df

    col_index = 0
    col_range = end_col - (start_col-1)
    cols_added = 0
    while col_index < col_range:
        sum_columns = df.iloc[:, start_col+col_index-1:start_col+col_index+n-1]
        sum = sum_columns.sum(axis=1)
        df[f'{name_prefix}nSum{n}_{cols_added}'] = sum
        col_index += n
        cols_added += 1

    return df

In [80]:
# Create the rowwise average of n days of observations for the specified range of columns; this range includes BOTH the start and end columns.
# Note: if you do this after using n_day_sum(), you will lose your sum columns and be left only with means. I'll fix it eventually.
# In the meantime, if you need both sums and means, do means first and then sums.
def n_day_mean(df, n=0, start_col=0, end_col=0, name_prefix=''):   

    df = n_day_sum(df, n=n, start_col=start_col, end_col=end_col, name_prefix=name_prefix)
    
    cols_avgd = 0
    num_sums = (end_col - (start_col-1))/n
    while cols_avgd < num_sums:
        sum_col_name = f'{name_prefix}nSum{n}_{cols_avgd}'
        mean_col_name = f'{name_prefix}nMean{n}_{cols_avgd}'
        df[sum_col_name] = df[sum_col_name] / n
        df.rename(columns={sum_col_name: mean_col_name}, inplace=True)
        cols_avgd += 1

    return df

In [143]:
# Create the local & global yearly average & attach it to the dataframe.

def yearly_means(df, start_col=0, end_col=0, name_prefix=''):
    df = n_day_mean(df, n=240, start_col=start_col, end_col=end_col, name_prefix=name_prefix)
    
    query = df.groupby('year')[name_prefix + 'nMean240_0'].mean().reset_index()
    query.rename(columns={name_prefix + 'nMean240_0': name_prefix + 'ga_mean'}, inplace=True)
    
    df = df.merge(query, on='year')
    
    return df

### DataFrame Preparation

#### Training set:

In [178]:
del train_df
gc.collect()

3322

In [151]:
train_df = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/train.parquet')

In [155]:
train_df.drop(labels='real_year', axis=1, inplace=True)
train_df.reset_index(inplace=True)

In [ ]:
train_df.head()

In [156]:
# Get the beginning and end index of each variable
prefixes = ['pr_', 'rsds_', 'tasmin_', 'tasmax_', 'tas_']
starts_stops = {}

for prefix in prefixes:
    starts_stops[prefix + 'start'] = train_df.columns.get_loc(prefix + '0')
    starts_stops[prefix + 'stop'] = train_df.columns.get_loc(prefix + '239')

starts_stops

{'pr_start': 8,
 'pr_stop': 247,
 'rsds_start': 248,
 'rsds_stop': 487,
 'tasmin_start': 488,
 'tasmin_stop': 727,
 'tasmax_start': 728,
 'tasmax_stop': 967,
 'tas_start': 968,
 'tas_stop': 1207}

In [157]:
for prefix in prefixes:
    train_df = n_day_mean(train_df, n=30, start_col=starts_stops[prefix+'start'], end_col=starts_stops[prefix+'stop'], name_prefix=prefix)
    train_df = n_day_sum(train_df, n=30, start_col=starts_stops[prefix+'start'], end_col=starts_stops[prefix+'stop'], name_prefix=prefix)
    train_df = yearly_means(train_df, start_col=starts_stops[prefix+'start'], end_col=starts_stops[prefix+'stop'], name_prefix=prefix)

In [162]:
start = train_df.columns.get_loc('pr_0')
stop = train_df.columns.get_loc('yield')

train_df.drop(train_df.iloc[:, start:stop], axis=1, inplace=True)

train_df = pd.get_dummies(train_df, columns=['crop'])
train_df.set_index('ID', inplace=True)

train_df.head()

KeyError: 'pr_0'

In [165]:
train_df.head()

,year,lon,lat,texture_class,co2,nitrogen,yield,pr_nMean30_0,pr_nMean30_1,pr_nMean30_2,pr_nMean30_3,pr_nMean30_4,pr_nMean30_5,pr_nMean30_6,pr_nMean30_7,pr_nSum30_0,pr_nSum30_1,pr_nSum30_2,pr_nSum30_3,pr_nSum30_4,pr_nSum30_5,pr_nSum30_6,pr_nSum30_7,pr_nMean240_0,pr_ga_mean,rsds_nMean30_0,rsds_nMean30_1,rsds_nMean30_2,rsds_nMean30_3,rsds_nMean30_4,rsds_nMean30_5,rsds_nMean30_6,rsds_nMean30_7,rsds_nSum30_0,rsds_nSum30_1,rsds_nSum30_2,rsds_nSum30_3,rsds_nSum30_4,rsds_nSum30_5,rsds_nSum30_6,rsds_nSum30_7,rsds_nMean240_0,rsds_ga_mean,tasmin_nMean30_0,tasmin_nMean30_1,tasmin_nMean30_2,tasmin_nMean30_3,tasmin_nMean30_4,tasmin_nMean30_5,tasmin_nMean30_6,tasmin_nMean30_7,tasmin_nSum30_0,tasmin_nSum30_1,tasmin_nSum30_2,tasmin_nSum30_3,tasmin_nSum30_4,tasmin_nSum30_5,tasmin_nSum30_6,tasmin_nSum30_7,tasmin_nMean240_0,tasmin_ga_mean,tasmax_nMean30_0,tasmax_nMean30_1,tasmax_nMean30_2,tasmax_nMean30_3,tasmax_nMean30_4,tasmax_nMean30_5,tasmax_nMean30_6,tasmax_nMean30_7,tasmax_nSum30_0,tasmax_nSum30_1,tasmax_nSum30_2,tasmax_nSum30_3,tasmax_nSum30_4,tasmax_nSum30_5,tasmax_nSum30_6,tasmax_nSum30_7,tasmax_nMean240_0,tasmax_ga_mean,tas_nMean30_0,tas_nMean30_1,tas_nMean30_2,tas_nMean30_3,tas_nMean30_4,tas_nMean30_5,tas_nMean30_6,tas_nMean30_7,tas_nSum30_0,tas_nSum30_1,tas_nSum30_2,tas_nSum30_3,tas_nSum30_4,tas_nSum30_5,tas_nSum30_6,tas_nSum30_7,tas_nMean240_0,tas_ga_mean,crop_maize,crop_wheat
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,381.0,-122.25,48.25,9.0,340.79,186.110992,5.595,6.203751,0.000039,0.000021,1.345042e-05,8.468729e-08,0.000032,0.000079,0.000106,186.112543,0.001160,0.000631,0.000404,0.000003,0.000967,0.002370,0.003177,0.775505,0.382502,136.870081,191.293948,219.781605,242.461284,254.818718,151.495101,74.881226,51.993473,4106.102428,5738.818436,6593.448163,7273.838510,7644.56153,4544.853031,2246.436780,1559.804185,165.449429,164.708671,6.942697,7.928811,8.871894,11.987053,11.866305,11.098921,6.501516,4.286272,208.280920,237.864319,266.156829,359.611605,355.989136,332.967619,195.045471,128.588166,8.685434,9.083823,13.642634,16.293501,18.668148,23.428719,24.800181,20.568557,12.933806,12.152040,409.279022,488.805023,560.044433,702.861574,744.005431,617.056699,388.014190,364.561189,17.810948,18.822284,9.467053,11.505351,13.138007,16.688049,17.716921,14.957947,9.427319,7.618935,284.011597,345.160523,394.140198,500.641482,531.507631,448.738403,282.819582,228.568053,12.564948,13.369028,True,False
1,381.0,-122.25,48.75,9.0,340.79,186.110992,5.895,6.203752,0.000051,0.000027,1.764786e-05,5.689739e-08,0.000046,0.000084,0.000106,186.112568,0.001523,0.000813,0.000529,0.000002,0.001381,0.002533,0.003170,0.775510,0.382502,142.285625,192.716779,219.133948,243.091105,249.657706,148.111056,76.879776,52.145860,4268.568744,5781.503372,6574.018430,7292.733160,7489.73118,4443.331670,2306.393280,1564.375785,165.502732,164.708671,5.764505,6.831168,7.859135,11.353322,11.400753,10.314580,5.627832,3.113635,172.935165,204.935028,235.774047,340.599672,342.022585,309.437410,168.834960,93.409058,7.783116,9.083823,12.252521,14.822797,17.254651,22.530415,23.922998,19.876835,12.314081,10.676568,367.575624,444.683901,517.639526,675.912445,717.689942,596.305054,369.422424,320.297028,16.706358,18.822284,8.323495,10.412573,12.028075,15.995186,17.167044,14.274389,8.577579,6.229894,249.704864,312.377196,360.842254,479.855591,515.011325,428.231659,257.327362,186.896820,11.626029,13.369028,True,False
2,381.0,-122.25,49.25,9.0,340.79,184.934006,3.023,6.164566,0.000083,0.000047,2.562528e-05,2.145416e-06,0.000080,0.000147,0.000176,184.936978,0.002490,0.001422,0.000769,0.000064,0.002396,0.004403,0.005285,0.770641,0.382502,134.314598,184.092455,205.268673,227.994068,234.685757,136.961652,68.465889,47.837171,4029.437933,5522.773651,6158.060181,6839.822034,7040.57271,4108.849570,2053.976681,1435.115119,154.952533,164.708671,4.732346,6.126607,7.510077,11.510121,11.799174,10.316551,5.470477,2.136389,141.970368,183.798218,225.3

#### Test set:

In [15]:
cols_to_select = ['crop', 'year', 'lon', 'lat',
                  'nMean30_0', 'nMean30_1', 'nMean30_2', 'nMean30_3', 'nMean30_4', 'nMean30_5', 'nMean30_6', 'nMean30_7',
                 'nSum30_0', 'nSum30_1', 'nSum30_2', 'nSum30_3', 'nSum30_4', 'nSum30_5', 'nSum30_6', 'nSum30_7']

test_df = pr_m[cols_to_select]
test_df.reset_index(inplace = True)
test_df = test_df.merge(sc_m, on=['ID'])

In [16]:
del pr_m, sc_m
gc.collect()

0

In [17]:
test_df = pd.get_dummies(test_df, columns=['crop'])
# need to make ID the index again

In [18]:
test_df.head()

,ID,year,lon,lat,nMean30_0,nMean30_1,nMean30_2,nMean30_3,nMean30_4,nMean30_5,nMean30_6,nMean30_7,nSum30_0,nSum30_1,nSum30_2,nSum30_3,nSum30_4,nSum30_5,nSum30_6,nSum30_7,texture_class,co2,nitrogen,crop_maize
0,349719,420.0,-122.25,48.25,0.000027,0.000012,0.000006,0.000015,2.786551e-06,0.000052,0.000056,0.000065,0.000821,0.000362,0.000187,0.000439,0.000084,0.001556,0.001687,0.001963,9.0,418.06,186.110992,True
1,349720,420.0,-122.25,48.75,0.000031,0.000012,0.000007,0.000016,3.054733e-06,0.000051,0.000066,0.000072,0.000942,0.000368,0.000205,0.000485,0.000092,0.001530,0.001984,0.002161,9.0,418.06,186.110992,True
2,349721,420.0,-122.25,49.25,0.000056,0.000025,0.000017,0.000029,8.828045e-06,0.000094,0.000122,0.000120,0.001674,0.000742,0.000513,0.000885,0.000265,0.002807,0.003671,0.003607,9.0,418.06,184.934006,True
3,349722,420.0,-119.75,47.75,0.000005,0.000004,0.000003,0.000006,2.145821e-06,0.000012,0.000002,0.000021,0.000149,0.000113,0.000077,0.000179,0.000064,0.000360,0.000062,0.000640,9.0,418.06,186.110992,True
4,349723,420.0,-116.75,43.25,0.000019,0.000014,0.000002,0.000001,6.854866e-07,0.000023,0.000006,0.000031,0.000567,0.000407,0.000064,0.000042,0.000021,0.000699,0.000195,0.000923,9.0,418.06,186.110992,True


### Model Setup

In [170]:
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import TimeSeriesSplit, HalvingGridSearchCV, HalvingRandomSearchCV

In [167]:
# Set up the metrics: root mean squared error & r squared (actual evaluation is on mean & median r squared; might set up custom metric for those)
scoring = ['neg_root_mean_squared_error', 'r2']

In [168]:
# Set up the training sets
X = train_df.drop(labels=['yield'], axis=1)
y = train_df['yield']

# Make sure columns line up (necessary once making predictions to submit)
# X_train, X_test = X_train.align(test_df, join='left', axis=1)

In [171]:
# Run the Decision Tree model and get the statistics on validation set using 5-fold cross validation (test set is online)
d_tree = DecisionTreeRegressor()
tscv = TimeSeriesSplit(n_splits=5)

# Set up grid of hyperparameters to search
params = {
    'max_depth': [int(x) for x in np.linspace(60, int(np.log2(len(X_train))), num=4)],
    'min_samples_split': [2, 10, 20],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [40, 60, 80]
}

grid_search = HalvingGridSearchCV(estimator=d_tree, param_grid=params, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1)
grid_search.fit(X, y)

best_params = grid_search.best_params_
print(f'Decision Tree best hyperparameters: {best_params}')

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Decision Tree best hyperparameters: {'max_depth': 46, 'max_features': None, 'max_leaf_nodes': 40, 'min_samples_split': 20}


In [173]:
d_tree = DecisionTreeRegressor(**best_params)
# tscv = TimeSeriesSplit(n_splits=5)

d_tree_scores = cross_validate(d_tree, X, y, scoring=scoring, return_train_score=True, cv=tscv, n_jobs=-1)
sorted(d_tree_scores.keys())

print('Decision Tree Results:')
print(f'Mean Test RMSE: {round(d_tree_scores['train_neg_root_mean_squared_error'].mean(), 2)}')
print(f'Mean Train RMSE: {round(d_tree_scores['test_neg_root_mean_squared_error'].mean(), 2)}')
print(f'Mean Test R2: {round(d_tree_scores['test_r2'].mean(), 2)}')
print(f'Mean Train R2: {round(d_tree_scores['train_r2'].mean(), 2)}')

Decision Tree Results:
Mean Test RMSE: -1.43
Mean Train RMSE: -1.39
Mean Test R2: 0.61
Mean Train R2: 0.72


In [175]:
# Run the Random Forest model and get the statistics on validation set (test set is online)
rand_f = RandomForestRegressor()
# tscv = TimeSeriesSplit(n_splits=3)

# Set up grid of hyperparameters to search
params = {
    'max_depth': [int(x) for x in np.linspace(60, int(np.log2(len(X_train))), num=4)],
    'min_samples_split': [2, 10, 20],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [40, 60, 80]
}

rand_search = HalvingRandomSearchCV(estimator=rand_f, param_distributions=params, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=-1)
rand_search.fit(X, y)

best_params = rand_search.best_params_
print(f'Random Forest best hyperparameters: {best_params}')

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 108 is smaller than n_iter=104744. Running 108 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Random Forest best hyperparameters: {'min_samples_split': 2, 'max_leaf_nodes': 40, 'max_features': None, 'max_depth': 18}


In [176]:
best_params = rand_search.best_params_
rand_f = RandomForestRegressor(**best_params)
# tscv = TimeSeriesSplit(n_splits=5)

rand_f_scores = cross_validate(rand_f, X, y, scoring=scoring, return_train_score=True, cv=tscv, n_jobs=-1)

In [177]:
print('Random Forest Results:')
print(f'Mean Test RMSE: {round(rand_f_scores['train_neg_root_mean_squared_error'].mean(), 2)}')
print(f'Mean Train RMSE: {round(rand_f_scores['test_neg_root_mean_squared_error'].mean(), 2)}')
print(f'Mean Test R2: {round(rand_f_scores['test_r2'].mean(), 2)}')
print(f'Mean Train R2: {round(rand_f_scores['train_r2'].mean(), 2)}')

Random Forest Results:
Mean Test RMSE: -1.4
Mean Train RMSE: -1.37
Mean Test R2: 0.61
Mean Train R2: 0.74
